# Frequencies and Cooccurrences

This notebook will teach you how to do some very basic text processing with the goal of extracting word frequencies as well as their cooccurrences!


## Know thy data

The data we will be working on is a 1 million word sample from the Polish Araneum corpus, "araneum.sample". Before you start hacking together your script, it is always a good idea to check what your data actually look like. You should be able to use your terminal for this (cmder on windows), just type "head -n 20 /path/to/the/araneum.sample" and hit enter, which will show you the first 20 lines of the file and should look something like this:

```
<doc did="00000000" length="1k-10k" crawl_date="2013-10-25" url="http://www.fundacjadzieciom.pl/fundacja.html,149,5" langdiff="0.37">
<gap chars="629" />
<p>
<s>
Nadmierne       nadmierny       Aj      adj:sg:nom:n:pos        1
napięcie        napięcie        Nn      subst:sg:nom:n  1
grupy   grupa   Nn      subst:sg:gen:f  1
mięśni  mięsień Nn      subst:pl:gen:m3 1
powoduje        powodować       Vb      fin:sg:ter:imperf       1
nieprawidłowe   nieprawidłowy   Aj      adj:sg:nom:n:pos        1
ustawienie      ustawić Vb      ger:sg:nom:n:perf:aff   1
w       w       Pp      prep:loc:nwok   1
stawie  staw    Nn      subst:sg:loc:m3 1
<g/>
.       .       Zz      SENT    1
</s>
<s>
Mięśnie mięsień Nn      subst:pl:nom:m3 1
hipertoniczne   hipertoniczne   Aj      adj:pl:nom:m3:pos       0
mają    mieć    Vb      fin:pl:ter:imperf       1

```

There's a couple of things to note here:
* Some of the lines start with "<" - these are meta data, they specify where documents, paragraphs and sentences begin and end in the corpus and provide some additional information like when the document was obtained and where from.
* The other lines all consist of five parts which are aligned. That tells us that the elements in each line are separated by tabs, which we will need to know if we want to extract this information from each line.
* We can also take a good guess as to what these five elements are, more specifically they seem to consists of the word as found in the corpus, something like a lemma, a part-of-speech, a morphologically more detailed tag and finally a 1 or a 0.

Assuming that these observations will hold throughout our sample (they probably will and if not we will hopefully receive an error rather than have that go unnoticed, though that will happen eventually), this is all we need to start extracting interesting things from it! You should be excited right now.


## Extract vocabulary

Let's start by activating our conda environment - on Linux or Mac just enter "source activate YourEnvironment" in the terminal, on Windows enter "activate YourEnvironment" into cmder. Then we can start the python interpreter by entering "ipython3".

The first thing we will do is create a list of all the words in our sample. For that we will need two things: access to the sample file and some way to keep track of the vocabulary. The file can be provided to python like so:

In [2]:
file_path = "../data/araneum.sample"
sample = open(file=file_path, mode="r")

This tells python that we want it to open our file and to do so in reading mode (as opposed to writing). To store our vocabulary we will use a list for now:

In [3]:
vocabulary = list()

We can verify that we have opened the right file by reading the first line from it, which should be identical to the first line we saw using the head command in the terminal. The file object (sample) comes with a method that allows to read one line at a time and we will store that first line in a variable "first_line" and display it by just entering its name:

In [4]:
first_line = sample.readline()
first_line

'<doc did="00000000" length="1k-10k" crawl_date="2013-10-25" url="http://www.fundacjadzieciom.pl/fundacja.html,149,5" langdiff="0.37">\n'

That looks about right. The quotation marks around the line tell us that this is a string of characters in python. There is one other difference to the terminal output, though - the line ends in "\n", which is the line break character, so it's how a program is told that it should start displaying text on a new line (like when you open a .txt file with notepad).

A quick note on that: Like any other data, text is stored as a long sequence of 0s and 1s in memory. When we tell python to open a file, all it will do is look up where the data associated with that file starts in memory and that's it. Now to extract a line, python must know when to stop reading from that sequence of 1s and 0s and readline() does so once it hits the pattern of 1s and 0s that encodes the newline character and then returns all the characters up to that point.

Now let's move on to a line that actually contains a word from the corpus. As we know from our first look at the corpus, the next four lines will also be meta data, so we can skip those. Because we only have the readline() method, we will still need to read each of those lines and every time we call readline() python will display the, so instead we will save the output of the first three lines to "\_" - the variable just named underscore is generally used for this purpose we will always use it, meaning that it gets overwritten on each of the next lines, which is fine, because we don't need its content:

In [5]:
_ = sample.readline()
_ = sample.readline()
_ = sample.readline()

The next line should be the first word in our corpus, so let's read it into a variable that we can work with and make sure it's what we want:

In [6]:
first_word = sample.readline()
first_word

'Nadmierne\tnadmierny\tAj\tadj:sg:nom:n:pos\t1\n'

Great, that's exactly what we want. Now we can actually see explicitly what we inferred from the output formatting of the head command earlier: There are tabs ("\t") in between the elements of the line. That's good, because we can use that to extract the individual elements. Remember again that this line is just a long sequence of symbols to python, it does not know that there are meaningful elements in it. So what we will do is split the line on those tabs, which should give us a list of five elements. From the quotation marks we learned that python understands that first_word is a string and conveniently string objects come with a method that allows to split it's content on a symbol we provide like so:

In [7]:
elements_first_word = first_word.split("\t")
elements_first_word

['Nadmierne', 'nadmierny', 'Aj', 'adj:sg:nom:n:pos', '1\n']

That looks good, but not perfect yet. Notice how the last element is not just the number 1 or 0, but still has the newline character attached to it. In general, when we work with text, chances are that we only care about what's written in each line and not about new line characters, so we will do that same thing we just did, but first get rid of the newline character. Aside from split(), strings in python also come with a method strip(), which looks at the left and the right of the string and removes symbols that are typically not wanted, in particular it will remove "\n"s and spaces:

In [8]:
first_word_clean = first_word.strip()
elements_first_word = first_word_clean.split("\t")
elements_first_word

['Nadmierne', 'nadmierny', 'Aj', 'adj:sg:nom:n:pos', '1']

Now that looks better. One more thing that might bother us is that the word is spelled with an uppercase n. To python, "Nadmierne" and "nadmierne" are not the same thing, so when we will want to count how of it appears in our sample later, these would result in seperate counts. So another common first step when working with text data is to treat everything as lowercase and once again string objects provide us with a method that will turn the string to lowercase - lower(). This is good moment to introduce what is called chaining, which is a neat feature of python's syntax that allows us to chain together method calls, such that the output of the first method is the input to the next and so forth. If we want to strip, lowercase and split our string, this is how we will do it from now on:

In [9]:
elements_first_word = first_word.strip().lower().split("\t")
elements_first_word

['nadmierne', 'nadmierny', 'aj', 'adj:sg:nom:n:pos', '1']

We're finally happy with our result. What we haven't talked about yet is how the output of our result has changed. Before we had a sequence of characters surrounded by quotation marks, which told us that we were dealing with a string, so we knew we can use the methods that this type of object provides. Now, we see that our output is enclosed in square brackets and each element, separated by commas, is again a string. This tells us that we are dealing with a list of strings and consequently we can use methods that are compatible with lists. One such function is the len() function, which returns the number of element that a list contains:

In [10]:
len(elements_first_word)

5

You will have noticed that we did not call this method using the .dot operator attached to the end of our list. For now we can say that there are in principle two types of functions, the ones that are part of the object (so they are defined in the code for strings or lists) and those that are "free-floating" and are fed all the arguments they operate over. We will get more into this when we talk about classes and objects later on in the course. In this particular case, though, len() is such a common operation in python for various objects, that it is added as syntactic sugar for convenience. The length method is actually part of the list object, but when we call len(elements\_first\_word), python understands that this method is special and calls it for us. The method that is called implicitly is actually this one:

In [11]:
elements_first_word.__len__()

5

Which sure enough returns the same result. These kinds of methods are always preceded and succeeded with two underscores when called on the object itself. That is good to know and might come up at a later point in the course, but for now we can forget about the details and just know that there are some methods that are called on the object itself with the dot operator and others are given the object as an argument.

What we do need to know, though, is that some objects come with a specific syntax for performing certain operations. Having a list is nice, but we will probably want to be able to access its elements. For lists, this is done by specifying the index of the element we want to access in squared brackets following the name of the list. Let's try that with the first element:

In [12]:
elements_first_word[1]

'nadmierny'

Now this looks good, but it's actually the first pitfall for python. Computer scientists tend to count differently from the rest of the population and start with 0 (you could say they count using non-negative integers rather than positive ones). So elements\_first\_word[1] is actually the second element of our list. To see that, let's try to access the fifth element the way we intuitively would:

In [13]:
elements_first_word[5]

IndexError: list index out of range

Ew. This throws an error and you can be absolutely certain that you will continue to see this one all the time. The error is fairly easy to understand, it tells us what type of error is (an IndexError, so something related to trying to index something, in our case the list) and that the problem is that the index is out of range, meaning that is is not actually contained in the set of indices that our list has.

Now that we know that, we can really access the first element of our list and add it to our vocabulary. Remember that we initialized the vocabulary as a list as well, so the next list method we encounter is what we will use to add an element to a list, append(), and then we can look at our vocabulary:

In [14]:
vocabulary.append(elements_first_word[0])
vocabulary

['nadmierne']

Looks good. Our vocabulary is faily small, containing only one element, and if we were to proceed this way there would be  little use in programming, having to read each line manually and do all of these things. So let's try to automate doing everything we have just done for all lines in our sample. We will write a function that does this for us, so we can call it on each line. Functions in python are defined using the def keyword, followed by the name we give our function, the inputs it needs in brackets and a colon. All lines preceding it that are indented are then interpreted as the body of the function, i.e. the part that does all the work. The return keyword is followed by what the output of the function is:

In [15]:
def process_line(line):
    elements = line.strip().lower().split("\t")
    word = elements[0]
    return word

Let's try it out on the line that contains the first word from before:

In [16]:
process_line(first_word)

'nadmierne'

Looking great. Now all we need is a way of doing this for every line in our sample file. There are several ways of doing this, but as iterating over an object is something that needs to be done all the time, python comes with a special syntax for it, too. To illustrate it, let's use the list of elements we have created for the first line of interest before and iterate over the elements, printing each one in uppercase:

In [17]:
for element in elements_first_word:
    element = element.upper()
    print(element)

NADMIERNE
NADMIERNY
AJ
ADJ:SG:NOM:N:POS
1


The syntax should be very straightforward to understand. In this for-loop, python will iterate over all the elements in elements_first_word, store the current element in the variable named element and execute the code that is indented under it. So this is equivalent to doing this:

In [18]:
element = elements_first_word[0]
element = element.upper()
print(element)

element = elements_first_word[1]
element = element.upper()
print(element)

# ...

NADMIERNE
NADMIERNY


So that's simple enough. This syntax will work for every object that can be iterated over (we will meet a few of those), more formally it works for every object that implements one of the special methods, namely the __iter__() method! Importantly, one such object is the file object, so we can iterate over the lines in our sample in the exact same way, instead of calling readline() manually.

We will now initialize our list again (so its empty) and go back to the beginning of the file, so we don't miss the lines we've read manually so far:

In [19]:
vocabulary = list()
sample.seek(0) # go to the 0th byte in the file, i.e. the beginning

0

Perfect. So let's process every line of the sample with our function:

In [20]:
for line in sample:
    word = process_line(line)
    vocabulary.append(word)

Runs through without complaining, always a good thing. Let's look at the first twenty words we added to the vocabulary:

In [21]:
vocabulary[0:20]

['<doc did="00000000" length="1k-10k" crawl_date="2013-10-25" url="http://www.fundacjadzieciom.pl/fundacja.html,149,5" langdiff="0.37">',
 '<gap chars="629" />',
 '<p>',
 '<s>',
 'nadmierne',
 'napięcie',
 'grupy',
 'mięśni',
 'powoduje',
 'nieprawidłowe',
 'ustawienie',
 'w',
 'stawie',
 '<g/>',
 '.',
 '</s>',
 '<s>',
 'mięśnie',
 'hipertoniczne',
 'mają']

Well, that's not optimal. We can immediately spot two problems. The first is that the meta data are included, which are clearly not words. And the other is that even if we did, the sentence tag is included twice. In other words we are adding non-words to our vocabulary and our vocabulary includes everything as often as it occurs in our sample!

We can fix that by making two small changes. The first deals with the meta data, which we will deal with in the processing function. Remember that we can count the number of element in a list and that it is only the lines containing words that have five elements which are separated by tabs. Let's use that to only return a word if the splitted line consists of five elements:

In [22]:
def process_line(line):
    elements = line.strip().lower().split("\t")
    if len(elements) == 5:
        word = elements[0]
        return word

Again, this syntax should be straightforward. The if keyword is followed by a condition, which is True if the length of the list is five and False otherwise. The following indented code is only executed when condition is met, otherwise the function does not do anything.

Let's try it out on the first line and the first word line:

In [23]:
process_line(first_line)

In [24]:
process_line(first_word)

'nadmierne'

Great. For first_line, which was meta data, we get no result returned, while for first_word, which contains a target, our function returns the word.

Next we will make sure that we only add to our vocabulary list in case our functions actually returns something and also make sure that the word is not already in the list, so we don't count anything twice! First we will set our file and our list back again and then add all the words:

In [25]:
vocabulary = list()
sample.seek(0)

for line in sample:
    word = process_line(line)
    if word and word not in vocabulary:
        vocabulary.append(word)

That went through. A quick word of explanation: We added another if condition, so vocabulary.append(word) is only executed if that condition is met. The condition is more complicated before and in particular includes and and. So really we are dealing with two condition here, the first being "if word" and the other being "if word not in vocabulary", the full condition is only True (i.e. the following code only gets executed) is both parts evaluate to True. When combining logical statements like this, you will need to be careful. Something like "False and False or True" could be interpreted as "(False and False) or True" or as "False and (False or True)", which have opposite truth conditions! When in doubt, google Python operator precedence.

The latter part should be self-explanatory, it simply checks whether word is in the list. For the first part, remember that process_line only returned a word for the targets and returned nothing otherwise. Nothing is not quite correct here, though. If a function returns nothing, it implicitly returns None. We can check that:

In [26]:
implicit_return = process_line(first_line)
implicit_return is None

True

"If object" is the pythonic way of checking whether an object contains anything. It will only be True if the object is not None and not 0, so for lines where the function did not return anything, this will evaluate to False and the if condition is False as well!

Let's look at the first words in our vocabulary:

In [27]:
vocabulary[0:10]

['nadmierne',
 'napięcie',
 'grupy',
 'mięśni',
 'powoduje',
 'nieprawidłowe',
 'ustawienie',
 'w',
 'stawie',
 '.']

Now that looks better. This is also a good moment to introduce another handy function which will sort our vocabulary for us:

In [28]:
vocabulary_sorted = sorted(vocabulary)

In [29]:
vocabulary_sorted[0:10]

['!',
 '!!',
 '!!!',
 '!!!!',
 '!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!!',
 '!?',
 '"',
 '""""',
 '""""""']

So this should be enough for today. That last output shows us that our vocabulary might not quite be what we would expect a vocabulary to look like. So you task now is to change the code in a way that we get something more useful. You should have everything you need for that, so I encourage you to think about how you could do this and then use google in case you need to do something that you don't yet know how to do.

Here's one solution. We will alter the process_line function to only return tokens that do not contain any punctuation:

In [30]:
import string

def process_line(line):
    elements = line.strip().lower().split("\t")
    if len(elements) == 5:
        word = elements[0]
        for character in word:
            if character in string.punctuation:
                break
        else:
            return word

The solution above introduced two new things, let's first talk about the import statement.

Like most programming languages, Python allows to use code that is not defined in the current script, but in other Python files (\*.py). A file that code can be imported from is called a *module* and several modules can be organized into a *package* - you can think of these as loosely corresponding to folders on your operating system (packages) and files within them (modules). Just like folders can contain subfolders, packages can contain subpackages. Code can be imported from a package, a subpackage or a specific module. This results in a hierarchical organization that allows to neatly organize code repositories in sensible ways and enforces clarity as to where a piece of code was defined (unlike the *Where does this random function come from?* in R).

There are two ways of importing a module or a package. The first is a full import, which the solution above uses. The statement translates to "import *everything* from the string module", which is one of the standard modules that Python comes shipped with. To stick with the explicitness, everything that is imported from the string module is now bound to the variable called string and we can access each individual definition from it with a dot. In the string module, there is a character string defined that is called "punctuation", which we can consequently access now as string.punctuation.

Because packages can become very big and we will usually only need some of the functionality that a package provides, we can also be more specific about what we want to import. As we are only interested in the punctuation string, we can import just that part of the module like so:

In [31]:
from string import punctuation

punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Notice how the string is now stored in the variable called puntuation, rather than string.punctuation. Because we are explicit in the import statement, we don't lose anything if we drop the prefix and won't waste time typing in the body of our code! We can get away with typing even less by making use of the last import functionality that Python offers which is assigning the imported module to a new (shorter) name:

In [32]:
from string import punctuation as p

p

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

While the amount of time saved typing that way is debatable (in particular, we want to give our variables informative names!), this functionality can be very handy when we would otherwise run into naming conflicts. Let's say we have defined our own variable punctuation somewhere earlier in the code and now import punctuation from string. Our own variable would be lost, because the name points only to the last thing it was assigned to. And while we could just have named our own variable differently, it is always possible that we need to import from several packages and that these happen to give their objects the same name, which is where this becomes really useful.

So that's imports sorted for now.

The other thing that was new in the solution are the *break* and the *else* statements in the for loop. The break statement "breaks out" of the for loop, meaning that the iteration will stop and not proceed to the next item(s) once a break is encountered. In our case, this happens in case any of the current word's characters is identified as a punctuation mark (in which case it will be found in the punctuation string, so the if condition evaluates to True and the break statement gets executed).

The else statement immediately following a for loop has a bit of an unfortunate name, because it is not executed instead of the for loop, but only if the for loop exits normally, so whenever the iteration continues to the end and is not broken out of (i.e. no break statement is executed). In the solution, this allows us to return a word only when none of its characters are punctuation, because otherwise the code indented under the else statement will not get executed and nothing is returned.

So, finally, let's try it out.

In [33]:
vocabulary = list()
sample.seek(0)

for line in sample:
    word = process_line(line)
    if word and word not in vocabulary:
        vocabulary.append(word)
        
vocabulary_sorted = sorted(vocabulary)
vocabulary_sorted[:20]

['0',
 '1',
 '10',
 '100',
 '1000',
 '100000',
 '1001',
 '1002',
 '10020',
 '10027',
 '1007',
 '1008',
 '101',
 '1017',
 '102',
 '1020',
 '1024',
 '1028',
 '103',
 '1031']

We seem to have gotten rid of the punctuation marks, but we're still left with numbers. This hints at a general problem that we might run into if we try to filter out things we don't want, as we will need to anticipate what those things might be and chances are we will miss something.

There is an alternative to this, which is to define the structure that a word must accord to and ignore everything that does not comply with said structure. More specifically, we will define an alphabet and then check whether we can generate the potential word using that alphabet - if we can't, it must contain something we don't want. We will go into more details later in the course, but the general way to do this is by using a regular expression, which is a template that specifies the rules according to which a character string can be generated. Python comes with a package *re* that provides regular expression functionality and here is what our final function will look like:

In [34]:
import re

polish_words = re.compile("^[a-pr-uwy-zA-PR-UWY-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ\-]*$")

def process_line(line):
    elements = line.strip().lower().split("\t")
    if len(elements) == 5:
        word = elements[0]
        if polish_words.search(word):
            return word

The second line creates the regular expression. There are a few that might not be immediately obvious:

1. The string string we provided starts with a ^ - This marks the beginning of a string. Regular expressions create a pattern that they look for and if we didn't use the ^, it would look for that pattern anywhere in the string and would find a match even if our word starts with characters that we don't want it to.
2. It ends in a dollar sign, which likewise marks the end of a word.
3. The actual alphabet is encapsulated in squared brackets. Everything in squared brackets is treated as a disjunction, so it will match any of those characters *once* rather than the sequence.
4. There are dashes in between some of the letters, which is a short hand for providing everything between those letters (including), so a-p means abcdefghijklmnop.
5. At the very end, there is a backslash followed by another dash. As we have just established, the dash has a special meaning in the regular expression specification, so if we want to match an actual dash in the word we are looking at, then we have to *escape* the dash, which is what the backslash does.
6. The squared brackets are followed by a kleene star. This is another special symbol for regular expressions and means "repeat the thing in front of me any number of times" - this is what completes the regular expression, in the brackets we define all the symbols allowed, the cleene start says we can have any number of those in sequence and the hat and the dollar sign says that this pattern needs to cover the full string (our word) and not just parts of it.

Our regular expression (polish_words) can now be used to look for the specified pattern in a given string, using its search function. Let's try it out on two examples:

In [35]:
polish_words.search("być")

<_sre.SRE_Match object; span=(0, 3), match='być'>

In [36]:
polish_words.search("Eiffel65")

That's looking good. The second test involves ordinals, which we have not specified in our template and consequently the regular expression can not generate this string and does not return anything. Now let's see what our vocabulary will look like:

In [37]:
vocabulary = list()
sample.seek(0)

for line in sample:
    word = process_line(line)
    if word and word not in vocabulary:
        vocabulary.append(word)
        
vocabulary_sorted = sorted(vocabulary)
vocabulary_sorted[:20]

['-',
 '--',
 '-a',
 '-am',
 '-amperowe',
 '-benzylopiperazyna',
 '-bitowym',
 '-brzucha',
 '-ch',
 '-chlorofenylopioperazyna',
 '-ciu',
 '-cylindrowa',
 '-cylindrowego',
 '-cylindrowej',
 '-cylindrowych',
 '-czy',
 '-dla',
 '-dniowe',
 '-dniową',
 '-do']

It looks better, though arguably including the dash might not have been such a great idea after all. Be that as it may, we will move on.

If you've run through all the code so far, you might have noticed that creating the vocabulary this way takes quite a while to run, while everything else has returned instantly. Now even though we are dealing with a million lines and doing a couple of operations on each of them, this is taking a lot longer than it should and the reason is that a list is the wrong data structure to use here. Instead of just providing the right data structure, we will use this opportunity to go into some detail about what is happening in the background and why it is so slow, as understanding this will come in quite handy later on, even if it is a bit off track right now.

Recall that what our code does is it creates a list object and then, for each word in our sample, it does two things - it (1) searches for that word in the list and (2) appends it otherwise. Lists are generally a very useful data structure and part of virtually every programming language. We will contrast lists from arrays to understand why lists are implemented the way they are and how that results in a bad fit to our task requirements to then eventually move on to a better suited data structure - a set.

Let's first talk about arrays. An array is a way of storing a collection of items *of the same type*, like a list of numbers (list in the natural language sense, not in the data structure sense!). The inner workings are very simple, we specify in advance how many numbers we want to store and the language will reserve a block of memory that can hold exactly that many numbers. Say our list of numbers is actually integers. Integers are often stored with 32 bits (4 bytes), so a sequence of thirty-two 1s and 0s, with the first bit containing the sign and the remaining 31 containing the number in binary. If our list consists of 100 integers, we will need 32000 bits. So the language will reserve that and then we can put our integers into the array, so we can say something like our_array[5] = 100. What happens now and what makes arrays really fast for accessing a specific index is that, because we know how much memory each element takes, we can figure our easily where *any* element is located. For accessing the fifth element, we know that the first four elements require 4\*32=128 bits. So clearly, the fifth element will be stored in the bits from 129 to 160, which can then be set to 00000000 00000000 00000000 01100100. In other words the time it takes to access an element is independent of the number of elements in the list (constant time). Iterating through the array is equally straightforward, all that is required is returning the contiguous block of memory in steps of 32 bits.

So arrays allow us to easily iterate, quickly access individual elements and be very economical, as we really only store the actual bits for the integers and have very little overhead (basically just the information that this variable is a list, how many elements it has in total, the type of the element stored and the memory address pointing to the beginning of the reserved block of memory).

One drawback is that we can only store elements of the same type. Clearly, calculating where anything is located in our block of memory wouldn't work if some of the stored elements require 32 bits, while others require only 8 bits, let alone dynamically sized objects like a string (which can be of any length and consequently size). Another is that adding an element will be comparably slow, because we can only reserve a specific amount of memory and now that we need more the memory right after that block might be used by something else - we might be lucky and there is more memory available after the reserved block (in which case we could simply reserve the next 32 bits), but if we're not lucky we need to reserve a new block of memory somewhere else and move all the stored elements there. And if we were to delete or insert something in the middle of our array, then even if there is extra memory space, we will need to move all the elements after the insertion or deletion point one step. If the elements that are stored in the array are big objects, then moving them can be quite costly.

Now we can contrast this with lists. A classical way of implementing a list would be what is called a linked list. In a linked list, no block of memory is specified in advance and the variable name we have given to our list would not point to the beginning of the memory block, but instead point to the first element of the list. Let's say we still have our list of integers, so the variable would contain the memory address of the first number in the list. Except that what is stored at that memory location in a linked list is a bit more elaborated than it was for the array. In particular it will contain not only the integer itself, but one more thing - a link to the *next* element in the list (again just the memory address). That element in turn provides a link to the one that follows it and so forth, until the last element points to a special end of the list element.

Let's see what that does for us. Iterating through the list is still easy, it just involves returning each integer representation and then following the link to the next element until the end is encountered. Appending an element simply involves changing the link of the last element to the appened one and pointing from there to the end of the list. Deleting only requires to change the link from the element before the deleted one to point to the one after it instead. And insertion requires two changes, so that the element before points to the new element and the new element points to the next one. In other words no contiguous block of memory needs to be allocated, so the number of elements does not need to be specified in advance, there is no reason for all elements to have the same size and nothing ever needs to be moved.

Linked lists also have some very serious drawbacks. In particular we cannot calculate where any element is located. To access the 100th element, we need to start at the first one and then follow 99 links to get to the target. For a large number of elements, this will be really slow. For appending to the list, in theory this would also hold (we would have to go through all elements), but a more clever implementation will not only remember the first, but also the last element, which makes appending fast and constant in time. Keeping links not only to the next, but also to the previous element makes traversing the list faster as well, but the general problem remains that accessing and changing the list will get significantly slower the bigger it gets.


This implementation also comes with the same drawbacks as the regular array, namely that the number of elements has to be chosen in advance, so that memory for the pointers can be allocated accordingly. But because pointers are cheap (in terms of memory size), Python can be clever about it and over-allocate, so that when a list is created with two elements, it actually allocates a block that can store the memory adresses for four elements. And once that is exceeded, it will reallocate so that it has now room for 8 elements. The overallocation is proportional to the size of the list, so that as the list grows bigger, memory for ever more addiIn Python, unlike what their name would suggest, lists are not actually imlemented as linked lists. Instead they are arrays. Importantly, they are not arrays of the stored objects themselves, but arrays of pointers to the objects (pointers are what memory locations are usually termed in this context). This has two huge advantages. The first is that we know the size of a pointer (8 bytes on 64-bit operating systems), so just like with a regular array we can calculate where any element's memory address is stored, which means that accessing that element is just as quick as in a regular array (+ following that one link to the memory location). The other advantage is that the stored elements can be of any size and live anywhere in memory, because all that is stored is the pointer, which always has the same size.
tional elements will be over allocated. The pattern for the first allocation looks like this: 0, 4, 8, 16, 25, 35, 46, 58, 72, 88, ...

What this results in is that the time it takes appending to the list amortizes, meaning that if we append a lot to a list, the number of times it will need to be resized decreases, which leads to constant time for appending.

Okay. So we know what a list looks like. Now think about what the problem is in our task before you continue reading.

The problem should be obvious. Appending is fast, so that can't be it. But how can Python know whether the list contains any given item? It will have to bite the bullet and compare each and every element in the list to the item in question. As our vocabulary grows, the time it takes to do that will also grow - exponentially! Our example only goes through a million candidates and the vocabulary size is some fifty thousand by the end - on a real corpus, we will deal with two orders of magnitude more data - our program might take weeks to complete. Clearly, we need another data structure, as lists and arrays will have the same problem here.

The data structure we need is a set. Just like in mathematics, a set is an *unordered* collection of *unique* items. Let's see it in action and then talk about it some more.

In [38]:
vocabulary = set()
sample.seek(0)

for line in sample:
    word = process_line(line)
    if word and word not in vocabulary:
        vocabulary.add(word)
        
vocabulary_sorted = sorted(vocabulary)
vocabulary_sorted[:20]

['-',
 '--',
 '-a',
 '-am',
 '-amperowe',
 '-benzylopiperazyna',
 '-bitowym',
 '-brzucha',
 '-ch',
 '-chlorofenylopioperazyna',
 '-ciu',
 '-cylindrowa',
 '-cylindrowego',
 '-cylindrowej',
 '-cylindrowych',
 '-czy',
 '-dla',
 '-dniowe',
 '-dniową',
 '-do']

This returned a lot faster, didn't it. The output is the same and most of the syntax is as well. Being a type of collection, sets allow iteration, sorting and the addition of elements. Unlike a list, however, because sets are unordered, iterating over them will return the collected elements in *some* order, but not in the order in which they have been added to them. And even though we can use the sorted function on a set, the object returned by that function (vocabulary_sorted) is a list, because sets, by definition, cannot have order. And because they don't have order, appending (as in add to the end) does not make sense and consequently they provide an .add() function instead.

How come sets are so fast here? We have talked about implementational details quite a lot so far, but sets are so clever that we will should understand how they work as well.

Remember that the problem before was that we had to go through the whole list in order to verify that an item is not included already. Imagine that instead we stored our vocabulary in several sublists, so that each list would contain only a fraction of the vocabulary. Let's call these sublists buckets. If we now knew what bucket any given item would be in, had we seen it, we would only need to go through that bucket and none of the others, cutting the number of elements we will have to check considerably. Try to think of a way you would do that (without worrying about how you could implement it).

Well, we are dealing with words. And as we have made sure of with our regular expression, we know what letters these can potentially consist of. So a simple way of realizing our idea would be to have a bucket for each letter of the alphabet and then put every word in that bucket that corresponds to the letter that the word starts with. And whenever we want to know whether we have seen a word, we will only need to check all the words we know that start with the same letter. This *would* work. But we can be yet a bit more clever about it.

If we take a step back, then our solution involved a very fast mapping from a large set of things (the words) to a much smaller set (the letters they start with) and a bucket for each of the latter (letter?). Let's think about what could be potentially suboptimal about our specific approach. How is anything in language distributed? Definitely not uniformly. If words are very likely to begin with some letters and very unlikely to begin with others, then the time we save this way will be greatly reduced, because we will spend most time searching through the high-frequency buckets and those buckets will also contain the most elements. That's not great. And even if it were distributed uniformly, it would divide the number of operations required by the number of elements in the alphabet, but as the number of items we store increases, this advantage will vanish. Finally, our solution is specific to the alphabet (we could solve that by creating a bucket for any unicode character) but more importantly it is also specific to character strings. What if we want to keep track of numbers too? What if we have ducks? What if we want to keep track of ducks?

So we want a different mapping function. One that is fast (if it takes forever to figure out which bucket to look into, then we might as well spend that time looking through the whole list), works on a range of different types of data, maps it onto an appropriate range of values and is as uniformly distributed in its range as possible, meaning that when we feed an arbitrary item to the function, then we should have equal probability of seeing each possible result (and look into the corresponding bucket) and there should be enough results to keep the size of the bucket appropriately small. Luckily, there is a type of function that is the perfect fit for our needs - hash functions.

We will not bother understanding the (super interesting!) details of how these work, but they have the properties we need. They take an arbitrarily sized input of various types and uniformly map them to an appropriate fixed range of values - the input's *hash*. Hashing functions are vital to cryptography, as they, for example, allow to verify that a password entered into a system is the right one (by comparing its hash to the stored has on the system, meaning that the password itself will never need to be stored and cannot be leaked) and this comes with the neat side-effect that they are well understood, meaning that hashing for sets will be very efficient.

So that's basically all there is to know about the internal workings of sets. They come with a hashing function and a number of buckets, so that every input gets hashed and stored or looked up in the bucket for the input's hash. Similar to how lists get resized, Python will use a hashing function that is appropriate for the number of elements the set stores, so that once there are so many elements that the size of each bucket becomes too large (meaning that it will take too much time to go through it), it will switch to another hashing function with a bigger range and rehash all the values it has seen before. In the limit, this will result in constant time for lookups and insertions - it'll be very fast.


## Counting

Finding all the unique words (types) in our sample is great, but typically we will want a bit more than that. Sets won't get us very far if we are interested in the token frequency of each type, as they make no note of how often anything was encountered. Using a plain list with all the tokens had the information we need, but was too slow and very wasteful, as for counting there is no need of actually storing all of the token. What we need is something that tells us what types there are and for each type gives us the token frequency. Mathematically, what we are want is a multiset.

In programming languages, this functionality is provided by hash maps or hash tables. In Python this is called a *dictionary*. A dictionary is very similar to a set (the name hash map might have tipped you off) in that it uses hashes and buckets to keep track of the things it contains. But instead of storing inputs in a bucket corresponding to the input's hash, dictionaries store pairs of keys and values. The key corresponds to the input, just as the set. The value is the extra bit that a dictionary provides and it is something that can be stored in the dictionary and that is uniquely identified by the key. In other words we can provide the dictionary with a key and a value and then query the dictionary just with the key and it will return the value. In that way it is similar to a list, but instead of using an index, we give each value a unique identifier (like a name) that we can use to access it.

We can use dictionaries to extend our vocabulary example to now keep track of frequencies:

In [39]:
vocabulary = dict()
sample.seek(0)

for line in sample:
    word = process_line(line)
    if word and word not in vocabulary:
        vocabulary[word] = 1
        
vocabulary_sorted = sorted(vocabulary)
vocabulary_sorted[:20]

['-',
 '--',
 '-a',
 '-am',
 '-amperowe',
 '-benzylopiperazyna',
 '-bitowym',
 '-brzucha',
 '-ch',
 '-chlorofenylopioperazyna',
 '-ciu',
 '-cylindrowa',
 '-cylindrowego',
 '-cylindrowej',
 '-cylindrowych',
 '-czy',
 '-dla',
 '-dniowe',
 '-dniową',
 '-do']

A couple of things to note here.

* Dictionaries are intialized with dict().
* We add to the dictionary by providing the key in squared brackets (just like the index in a list) and specifying that the value associated with that key whatever we want (1 in this case).
* If we sort the dictionary, we get a list of the keys.

That's not quite what we want, we want the frequencies. Instead of getting a sorted list of the keys, we want a list of the items, i.e. key-value pairs. Here's how to do that:

In [40]:
vocabulary_sorted = sorted(vocabulary.items())
vocabulary_sorted[:20]

[('-', 1),
 ('--', 1),
 ('-a', 1),
 ('-am', 1),
 ('-amperowe', 1),
 ('-benzylopiperazyna', 1),
 ('-bitowym', 1),
 ('-brzucha', 1),
 ('-ch', 1),
 ('-chlorofenylopioperazyna', 1),
 ('-ciu', 1),
 ('-cylindrowa', 1),
 ('-cylindrowego', 1),
 ('-cylindrowej', 1),
 ('-cylindrowych', 1),
 ('-czy', 1),
 ('-dla', 1),
 ('-dniowe', 1),
 ('-dniową', 1),
 ('-do', 1)]

Sorting the dictionary items again resulted in a list. Just like sets, dictionaries are by their hashing nature not ordered. In the newest Python version, this is actually not the case anymore and the insertion order is preserved (and with Python 3.7 this is a reliable feature of the language rather than just supported), but we will act as if it is for now.

More importantly, when we sort the items, they are still sorted alphabetically by the key. If we are interested in what the most frequent items are, this will not be very helpful. The sorting function takes a sorting key as an optional argument "key". This argument expects a function, which is fed each item from the list to be sorted (the list of key-value pairs) and returns something that Python knows how to sort (so basically numbers and characters). Let's write a function that takes key-value pair and returns the value, so the list will get sorted by that.

(If you've looked at the output from the last cell closely enough, you may have a feeling that we will still have a problem. That's okay, let's do it anyway.)

In [41]:
def sorting_key(pair):
    key, value = pair
    return value

vocabulary_sorted = sorted(vocabulary.items(), key=sorting_key)
vocabulary_sorted[:20]

[('podołać', 1),
 ('radiem', 1),
 ('cewkami', 1),
 ('skrócie', 1),
 ('bolesne', 1),
 ('lazari-pawłowska', 1),
 ('innego', 1),
 ('hipoteką', 1),
 ('wypełnienia', 1),
 ('warianty', 1),
 ('niedokonywanie', 1),
 ('pzpr', 1),
 ('rączkę', 1),
 ('surowizny', 1),
 ('licencyjnej', 1),
 ('srodku', 1),
 ('samoregulator', 1),
 ('pegasus', 1),
 ('zwalczać', 1),
 ('miałbym', 1)]

Hm. So there was no error (good) and the items are definitely not sorted alphabetically anymore (good). But all the frequencies are 1 (not good). Our sorting key returned numbers and by default, the sorted() function will order in ascending order, so it will start with the smallest. Let's quickly verify that:

In [42]:
numbers = [5,2,3,4,1]
sorted(numbers)

[1, 2, 3, 4, 5]

sorted() has another optional parameter that tells it to reverse the sorting order:

In [43]:
vocabulary_sorted = sorted(vocabulary.items(), key=sorting_key, reverse=True)
vocabulary_sorted[:20]

[('podołać', 1),
 ('radiem', 1),
 ('cewkami', 1),
 ('skrócie', 1),
 ('bolesne', 1),
 ('lazari-pawłowska', 1),
 ('innego', 1),
 ('hipoteką', 1),
 ('wypełnienia', 1),
 ('warianty', 1),
 ('niedokonywanie', 1),
 ('pzpr', 1),
 ('rączkę', 1),
 ('surowizny', 1),
 ('licencyjnej', 1),
 ('srodku', 1),
 ('samoregulator', 1),
 ('pegasus', 1),
 ('zwalczać', 1),
 ('miałbym', 1)]

Huh. Same result.

This is actually expected. We've made a mistake in the way we kept track of the frequencies... and your next task will be to find it and figure out how to fix it!

Here's the solution:

In [46]:
vocabulary = dict()
sample.seek(0)

for line in sample:
    word = process_line(line)
    if word:
        if word not in vocabulary:
            vocabulary[word] = 1
        else:
            vocabulary[word] += 1
        
vocabulary_sorted = sorted(vocabulary.items(), key=sorting_key, reverse=True)
vocabulary_sorted[:20]

[('w', 24936),
 ('z', 13618),
 ('i', 11773),
 ('na', 9796),
 ('-', 8118),
 ('do', 7810),
 ('się', 6597),
 ('nie', 6057),
 ('o', 5180),
 ('jest', 4710),
 ('od', 4439),
 ('czy', 4368),
 ('to', 4257),
 ('że', 3529),
 ('przez', 3485),
 ('a', 3445),
 ('spółki', 3197),
 ('sygnatura', 2644),
 ('oraz', 2477),
 ('interpretacja', 2411)]

Now we're talking. Before we were only creating dictionary entries for unseen words and setting them to 1, so we never did any counting. Now we do the same thing, but for words we have seen, we increase the counter by 1. The *variable += 1* is a shorthand for *variable = variable + 1*!

As counting is such a common task, python comes with a convenience object that is useful here, the Counter from the collections module. It behaves mostly like a regular dictionary, but since it is clear that it will count stuff, it automatically creates entries for unseen items with a count of 0 and has a method for returning the most frequent items. Our code will look like this:

In [47]:
from collections import Counter

vocabulary = Counter()
sample.seek(0)

for line in sample:
    word = process_line(line)
    if word:
        vocabulary[word] += 1

vocabulary.most_common(20)

[('w', 24936),
 ('z', 13618),
 ('i', 11773),
 ('na', 9796),
 ('-', 8118),
 ('do', 7810),
 ('się', 6597),
 ('nie', 6057),
 ('o', 5180),
 ('jest', 4710),
 ('od', 4439),
 ('czy', 4368),
 ('to', 4257),
 ('że', 3529),
 ('przez', 3485),
 ('a', 3445),
 ('spółki', 3197),
 ('sygnatura', 2644),
 ('oraz', 2477),
 ('interpretacja', 2411)]

Counting tokens is very useful, but sometimes we are interested in counting more complex objects, say the token with their respective part-of-speech tags. Easy enough, you think, we will just make a list of the token and its tag and count those! Let's give that a try. First we'll need to change our processs_line function:

In [86]:
def process_line(line):
    elements = line.strip().lower().split("\t")
    if len(elements) == 5:
        word, _, tag = elements[0:3]
        if polish_words.search(word):
            return [word, tag]

Instead of returning just the token, the function now returns a list of the word and its tag. We've also learned that we can take the first three elements of the line (the slice of the list of elements, spanning from the position before the first element (0) to the position after the third element (3) - *elements[0:3]*) and assign them to three variable names at the same time (*word, _, tag = elements[0:2]*) - remembering that the second element of each line was the lemma that we don't care about. Explicitly assigning the elements of a collection to individual variables is called unpacking.

So let's tabulate these lists the same way as before:

In [49]:
vocabulary = Counter()
sample.seek(0)

for line in sample:
    word = process_line(line)
    if word:
        vocabulary[word] += 1

TypeError: unhashable type: 'list'

Well that didn't work. The error message is interesting, it tells us that the object we're trying to add to the dictionary (a list) is not hashable. Remember that sets and dictionaries internally use hashing to store their items. The problem with lists is that they are meant to be dynamic, so we would expect that a list used somewhere in a piece of code has a fair chance of getting modified somewhere else in the code. But if the list changes, its hash will also likely change. So if we have a dictionary with lists as keys (for example to keep track of some information about the list, such that dictionary\[my_list\] = some_information_about_my_list), then after modifying it, the dictionary will look into a different bucket when queried with our list. In other words dictionary\[my_list\] != dictionary\[my_list_later\]. So the dictionary has no way of telling whether it has seen a list before or not, if it has changed.

That's pretty bad. And it will generally hold of objects that we can modify. These objects are called *mutable* and sets and dictionaries will refuse to keep track of them. Lists are mutable. This is a good moment to point to a difference between strings and lists. So far, we have treated strings as basically being lists (of letters) with some extra functionality, so that methods for lists are also methods for strings. But that's not quite true. Obviously they differ in that we were able to put strings into sets and dictionaries, but not lists. The reason is that strings are *immutable*, i.e. once a string object is created, it cannot be modified.

Let's make this difference more clear with some examples. We will take a string and turn it into a list, so we have a list and a string object that contain similar data:

In [50]:
my_string = "itwasabee"
my_string_list = list(my_string)

print(my_string)
print(my_string_list)

itwasabee
['i', 't', 'w', 'a', 's', 'a', 'b', 'e', 'e']


Okay. Now what happens if we try appending?

In [51]:
my_string_list.append("!")
print(my_string_list)

my_string.append("!")

['i', 't', 'w', 'a', 's', 'a', 'b', 'e', 'e', '!']


AttributeError: 'str' object has no attribute 'append'

Appending to the list was successful. Appending to the string was not, python complains that the append() method does not exist in the string object.

Well, fine. That method does not exist on strings. Let's next try modifying one the elements of these, given that we can access them in a similar fashion:

In [52]:
my_string_list[6] = "p"
print(my_string_list)

my_string[6] = "p"

['i', 't', 'w', 'a', 's', 'a', 'p', 'e', 'e', '!']


TypeError: 'str' object does not support item assignment

Again, we only get to modifying the list, but not the string. This time python is a bit more explicit and tells us that item assignment is not allowed for string objects. Now you may be wondering how come we were able to modify strings before, since we did strip off whitespaces from our lines and put them to lowercase. If you go back and look at the code, you might see how come we were able to modify our strings. (Hint: We didn't.)

The methods we have used for string did not actually modify the string they were called on, but instead left that object the way it was and returned a new object, which we then used instead. If we do my_string.strip().lower(), then my_string is not set to lowercase and retains all of its trailing whitespaces. But those functions *return a new string*, which we then assigned to the some variable name and continued working with.

Earlier we've learned that some functions are called directly on the objects they operate over (with a dot at the end of the variable pointing to the object) and others are given the object as an argument. We will still not go into detail on that yet, but now we've learned about another difference that is absolutely crucial for programming: some functions modify some of their arguments and others return a modified copy of them instead.

In general, functions that leave their arguments unmodified and return whatever they do modify are much easier to reason over, so I generally prefer them, but sometimes it is simply not possible or computationally inefficient to retain two copies of a data structure, like when you have a large matrix and need to add some value to all of its elements - unless you still need to work with the original data, it is much more effcient to modify the values *in-place*. For that reasons, some functions even come in both flavours, like the sorting function:

In [54]:
sorted_string_list = sorted(my_string_list)
print(sorted_string_list)

my_string_list.sort()
print(my_string_list)

sorted_string = sorted(my_string)
print(sorted_string)

my_string.sort()

['!', 'a', 'a', 'e', 'e', 'i', 'p', 's', 't', 'w']
['!', 'a', 'a', 'e', 'e', 'i', 'p', 's', 't', 'w']
['a', 'a', 'b', 'e', 'e', 'i', 's', 't', 'w']


AttributeError: 'str' object has no attribute 'sort'

We can call the sorted() function on mutable and immutable objects, as it returns a new sorted list. The .sort() function, however, is again only defined for lists, but not for strings, as it modifies the object it is called on in-place, so my_string_list itself is now sorted and nothing was returned (except None, implicitly).

This difference may seem like a harmless side note. But mutability is what distinguishes [programming styles](https://stackoverflow.com/questions/602444/functional-declarative-and-imperative-programming/15382180#15382180) and the way python handles how things are stored in variables will mean that this is very likely to creep up on you at some point. To see why, let's briefly summarize what python does when we store something in a variable.

When we execute *x = 8675309*, python does not actually *store* the value 8675309 in x. Instead, the thing on the left of the assignment operator now points to the thing on the right. In this case, this means that python will create an integer object (8575309) and the variable x now points to that object. This difference becomes important when we next execute *also_x = x*. If python stored a value on assignment, then there would now be two integer objects, both with the value 8675309 and contained in one of the variables. But there aren't. There is only integer object and both variables point to it. This state of affairs leads to a very common pitfall in python. Let's play a little game.

Read the following code sections and try to predict the output of the line I commented out before you uncomment and run it:

In [55]:
x = 5
y = x
y += 2
# y
# x

In [56]:
l = [1,2,3,4]
k = l
k[3] = 6
# l

In [60]:
h = [1,3,4,2]
j = h
j.sort()
# j[1]
# h[1]

In [61]:
r = [1,3,4,2]
h = r.sort()
# r
# h

In [59]:
a = ["t","e","s","t"]
b = a[2]
b = "n"
# a

In [58]:
d = dict()
d["first"] = 1
d["second"] = 2

f = d
f["first"] = 3
# d["first"]

All of the fun. There is one bit, where this becomes particularly tricky and that when functions are initialized with default arguments. Take a look at this:

In [64]:
def my_append(my_list = [], thing="stuffs"):
    my_list.append(thing)
    return my_list

Out function is a custom append, it takes a list and an object and appends the object to the list. If no list is provided, it creates and empty list and if no thing is provided it appends "stuffs". Let's give a try.

In [69]:
some_list = ["my"]
some_list = my_append(some_list, "pretty")
some_list

['my', 'pretty']

That works fine. Let's give it a few more tries, making use of the default arguments.

In [70]:
new_list = my_append(some_list)
new_list

['my', 'pretty', 'stuffs']

In [71]:
some_list

['my', 'pretty', 'stuffs']

There's the first problem. The function mutates the argument we give it, so we end up changing the some_list and don't just get new_list back. But we knew that. Let's use both default arguments.

In [72]:
other_list = my_append()
other_list

['stuffs']

Looks good. Let's do it again.

In [73]:
yet_another_list = my_append()
yet_another_list

['stuffs', 'stuffs']

Huh. That's unexpected. Somehow my_append added "stuff" twice to the default empty list. The reason is that in python default arguments to functions are evaluated at the time the definition is executed and never re-initialized. That means that the empty list was actually created before we ever called the function and the last two calls that did not specify a list argument added to that default list. The way to avoid that is to create the empty list inside the function call, rather than specify it as a default argument:

In [78]:
def my_better_append(my_list = None, thing = "stuffs"):
    if my_list is None:
        my_list = []
    my_list.append(thing)
    return my_list

first_call = my_better_append()
second_call = my_better_append()

print(first_call)
print(second_call)

['stuffs']
['stuffs']


That's better. The thing to remember from this is that some data structures (like lists or dictionaries) are mutable, meaning they can be modified, while others (like strings and integers) are immutable, such that modifying them will always create a new object. Functions can modify the arguments they are given to, which can quickly lead to unexpected behaviour. Default arguments are evaluated when the function is created and not each time it is called.

Enough digression, let's return to our words with POS tags. So we have established now that we can't use lists, because they are mutable. So we will need another data structure that can hold multiple items, but is not mutable. In python, this will be a tuple. Tuples function very similar to a string, but can contain any type of objects, rather than just characters:

In [80]:
my_tuple = ("stuffs", 1)
print(len(my_tuple))
print(my_tuple[0])

my_tuple[0] = "notstuffs"

2
stuffs


TypeError: 'tuple' object does not support item assignment

Tuples are generally used to hold units of data of a known structure. Items in dictionaries are such a type of data, since we know that each item is a key-value pair or a (key,value)-tuple, that's why dictionary.items() returned a list of tuples. Our tokens and POS tags also form nice little pairs, so it makes sense to store them as tuples as well and then we can finally tally them:

In [87]:
def process_line(line):
    elements = line.strip().lower().split("\t")
    if len(elements) == 5:
        word, _, tag = elements[0:3]
        if polish_words.search(word):
            return word, tag

vocabulary = Counter()
sample.seek(0)

for line in sample:
    word_and_tag = process_line(line)
    if word_and_tag:
        vocabulary[word_and_tag] += 1

vocabulary.most_common(20)

[(('w', 'pp'), 24936),
 (('z', 'pp'), 13618),
 (('i', 'cj'), 11750),
 (('na', 'pp'), 9796),
 (('-', 'zz'), 8118),
 (('do', 'pp'), 7810),
 (('się', 'pt'), 6597),
 (('nie', 'pt'), 5978),
 (('o', 'pp'), 4981),
 (('jest', 'vb'), 4710),
 (('od', 'pp'), 4439),
 (('że', 'cj'), 3529),
 (('przez', 'pp'), 3485),
 (('a', 'cj'), 3430),
 (('spółki', 'nn'), 3197),
 (('czy', 'pt'), 2946),
 (('sygnatura', 'nn'), 2644),
 (('oraz', 'cj'), 2477),
 (('interpretacja', 'nn'), 2411),
 (('indywidualna', 'aj'), 2390)]

That's works and it's nice, but what if we wanted to see all the POS tags that a given form occurs with? We would have to search through all the entries manually. We can do better if we don't use tuples (sorry, we had introduce them somehow) and instead just use dictionaries. While a dictionary key can only be an immutable object, values can be anything. In particular, a value could be yet another dictionary.

Aaaand your next task will be to implement that idea!

In [ ]:
# put your code here